# Neuromorphic Software: Phase-Encoding

Phase encoding is a biologically-inspired method of converting input data (like images) into temporal spike patterns, similar to how biological neurons process information. In neuromorphic computing, it plays a crucial role by:

1. Converting static input (pixel values) into dynamic temporal patterns
2. Mimicking the way biological sensory neurons encode information through spike timing
3. Enabling efficient processing of information in neuromorphic hardware

This implementation combines:
- A fixed bio-inspired phase encoder that transforms input data into spike-timing based representations
- A learnable MultiLayerPerceptron for classification that processes these phase-encoded patterns

The encoder uses phase differences and spike timing to create a rich representation of the input data, while preserving spatial relationships through a reference phase calculation. This makes it particularly suitable for pattern recognition tasks like digit classification.


## Notebook setup

Imports, Random Seeding, Device Setup - Training time varies based on hardware: approximately 20 minutes on CPU, and significantly faster on GPU.


In [ ]:
import random
from typing import Callable

import numpy as np
import pandas as pd
import plotly.express as px
import torch
import torch.nn as nn
import torch.optim as optim
from IPython import display
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

In [ ]:
# to ensure reproducibility, we should set the random seed consistently
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)  # for multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [ ]:
# --- Device Configuration ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Experiment Setup

This section defines key parameters for the experiment:

- Classification task parameters (number of classes, data augmentation limits)
- Neural network architecture parameters (layer dimensions, dropout rate)
- Training hyperparameters (epochs, batch size, learning rate, early stopping patience)

These parameters control the behavior of both the phase encoder and the classifier training process.


In [ ]:
# --- 0.0 Define Experiment Parameters ---
NUM_CLASSES = 10  # 10 digits in MNIST Dataset
MAX_ROTATION_DEGREES = 35  # Rotate images by up to 35 degrees
MAX_TRANSLATION = 0.1  # Translate images by up to 10%

# --- 0.1 Define Classifier Model Parameters ---
L1_OUTPUT_DIMS = 784
L2_OUTPUT_DIMS = 512
DROPOUT_PROB = 0.5  # Randomly "mute" a proportion of input neurons

# --- 0.2 Define Training Parameters ---
NUM_EPOCHS = 5
BATCH_SIZE = 64
LEARNING_RATE = 0.001
PATIENCE = 3

## Loading and Preprocessing MNIST Data
- Defines image transformations to be applied to the training and testing data, respectively.
- The training data is augmented with random rotations and translations.
- Both datasets are converted to PyTorch tensors and normalized.
- Normalization is crucial as it helps neural networks converge faster and perform better by ensuring all input features are on a similar scale and centered around zero.
- Loads the MNIST dataset.
- The root argument specifies where to store the data, train=True indicates the training set, and download=True downloads the data if it's not already present.
- Splits the training data into training and validation sets.
- This is important to evaluate the model's performance during training and prevent overfitting.
- Creates data loaders for the training, validation, and testing sets.
- These loaders handle batching and shuffling of the data, making it easier to feed into the model during training and evaluation.


In [ ]:
# --- 1. Load and Normalize MNIST ---
transform_train = transforms.Compose([
    transforms.RandomRotation(MAX_ROTATION_DEGREES),
    transforms.RandomAffine(degrees=0, translate=(MAX_TRANSLATION, MAX_TRANSLATION)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Mean and std deviation for MNIST
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Mean and std deviation for MNIST
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform_test)
# --- Split training data into training and validation sets ---
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

# --- 2. Dataloaders ---
batch_size = BATCH_SIZE
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Phase Encoding Setup

#### Phase Encoding Parameters:
- These variables (`N`, `omega_active`, `theta_thresh`, `omega_ref`, `n`, `kappa, x`) are parameters used in the phase encoding process.
- They control aspects of how the image data is transformed into a phase-encoded representation.

#### Input:

- $I$: Flattened image (a vector of pixel values)
- $N$: The X 8 Y dimensions of the image = N pixel count
- $\omega_{active}$: Active frequency parameter
- $x$: Spatial layout parameter (a vector) initialized to a range of values representing a phase gradient across a field of sensory neurons.

#### Parameters:

- $\theta_{thresh}$: Threshold phase (set to `0.0` in the code). When a Neuron's phase has rotated through $2\pi$ to $0$ we consider the Neuron has generated a Spike.
- $\omega_{ref}$: Reference frequency (set to `20 Hz` in the code).
- $n$: Scale factor (set to `4.0` in the code) that controls the spread of the reference phase calculation. Higher values compress the phase distribution.
- $\kappa$: Phase gradient coefficient (set to $2\pi$ in the code) that controls how strongly the spatial layout affects the reference phase calculation.


In [ ]:
# --- 3. Phase Encoding Parameters ---
N = 28 * 28
omega_active = torch.ones(N, dtype=torch.float32) * 2 * np.pi * 20.0
theta_thresh = 0.0
omega_ref = 2 * np.pi * 8.0
n = 4.0
kappa = 2 * np.pi
x_spatial_field = torch.linspace(0, 1, N, dtype=torch.float32)

EncoderFnType = Callable[[torch.Tensor, torch.Tensor, torch.Tensor], torch.Tensor]

### Phase Encoder Function

#### Calculation Steps:

1. **Initial Phase** ($\theta_{init}$) - Scales input pixel values (0-1) to phase range (0-2π): $$\theta_{init} = I \cdot 2\pi$$

2. **Phase Difference to Threshold** ($\Delta\theta$) - Calculates how far each initial phase needs to rotate to reach threshold: $$\Delta\theta = (\theta_{thresh} - \theta_{init} + 2\pi) \pmod{2\pi}$$

3. **Spike Time** ($t_{spike}$) - Converts phase difference into time domain using angular velocity: $$t_{spike} = \frac{\Delta\theta}{\omega_{active}}$$

4. **Reference Phase** ($\theta_{ref}$):
- Combines temporal coding (spike time) with spatial information $(x)$
- Normalized by factor n to control phase distribution: $$\theta_{ref} = \left(\frac{\omega_{ref} \cdot t_{spike} + \kappa \cdot x}{n}\right) \pmod{2\pi}$$

5. **Final Phase Difference** ($\phi$) - Computes phase difference between reference and threshold: $$\phi = (\theta_{thresh} - \theta_{ref} + 2\pi) \pmod{2\pi}$$

#### Output Encoding:
Final phase is encoded as a 2D vector using: $$[\cos(\phi), \sin(\phi)]$$
This representation:
- Preserves circular nature of phase
- Provides continuous, differentiable values
- Maintains equal magnitude across all phases

### Formal Definition:
$$ \text{encode\_image}(I, \omega_{active}, x) = [\cos(\phi), \sin(\phi)] $$
where
$$ \phi = (\theta_{thresh} - \theta_{ref} + 2\pi) \pmod{2\pi} $$


In [ ]:
# --- 4. Phase Encoder Function ---
def encode_image(img_flat: torch.Tensor,
                 omega_active_param: torch.Tensor,
                 x_param: torch.Tensor) -> torch.Tensor:
    theta_init = (img_flat * 2 * np.pi)
    delta_theta = torch.fmod(theta_thresh - theta_init + 2 * np.pi, 2 * np.pi)
    t_spike = delta_theta / omega_active_param
    theta_ref = torch.fmod((omega_ref * t_spike + kappa * x_param) / n, 2 * np.pi)
    phase_diff = torch.fmod(theta_thresh - theta_ref + 2 * np.pi, 2 * np.pi)
    return torch.cat([torch.cos(phase_diff), torch.sin(phase_diff)])

In [ ]:
# --- 5. Define Classifier ---
class PhaseClassifier(nn.Module):
    def __init__(self, num_classes: int):
        super(PhaseClassifier, self).__init__()
        self.layer1 = nn.Linear(N * 2, L1_OUTPUT_DIMS)
        self.layer2 = nn.Linear(L1_OUTPUT_DIMS, L2_OUTPUT_DIMS)
        self.layer3 = nn.Linear(L2_OUTPUT_DIMS, num_classes)
        self.dropout = nn.Dropout(DROPOUT_PROB)
        self.relu = nn.ReLU()

        # Initialize weights using Xavier/Glorot initialization
        nn.init.xavier_uniform_(self.layer1.weight)
        nn.init.xavier_uniform_(self.layer2.weight)
        nn.init.xavier_uniform_(self.layer3.weight)

        # Initialize biases to small constant values
        nn.init.constant_(self.layer1.bias, 0.01)
        nn.init.constant_(self.layer2.bias, 0.01)
        nn.init.constant_(self.layer3.bias, 0.01)

    def forward(self,
                batch: torch.Tensor,
                omega_active_param: torch.Tensor,
                spatial_layout: torch.Tensor,
                encoder_fn: EncoderFnType = encode_image
                ):
        # Flatten all images in batch at once
        batch_flat = batch.view(batch.size(0), -1)

        # Apply encoder function to all flattened images
        encoded_images = torch.stack([
            encoder_fn(img, omega_active_param, spatial_layout)
            for img in batch_flat
        ])

        encoded_images = self.dropout(encoded_images)
        x = self.relu(self.layer1(encoded_images))
        x = self.relu(self.layer2(x))
        x = self.dropout(x)
        logits = self.layer3(x)
        return logits


## Training Setup
- Blah

In [ ]:
# --- 6.1 Training Loop ---
def run_training(train_loader_param: DataLoader,
                 val_loader_param: DataLoader,
                 encoder_fn: EncoderFnType = encode_image) -> nn.Module:
    num_classes = NUM_CLASSES
    _model = PhaseClassifier(num_classes).to(device)
    optimizer = optim.Adam(_model.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss()

    print(_model)

    # --- Early Stopping Parameters ---
    best_val_loss = float('inf')
    patience = PATIENCE
    counter = 0

    num_epochs = NUM_EPOCHS
    loss = float('inf')

    # Move static tensors to device outside loop
    omega_dev = omega_active.to(device)
    x_dev = x_spatial_field.to(device)

    # Create an empty DataFrame to hold epoch and loss values
    df = pd.DataFrame({'Epoch': [], 'Loss': [], 'Validation Loss': []})
    epoch = 0
    for epoch in range(num_epochs):
        _model.train()
        loop = tqdm(train_loader_param, desc=f"Epoch {epoch + 1}/{num_epochs}", leave=False)
        for images, labels in loop:
            images_dev, labels_dev = images.to(device), labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = _model(images_dev, omega_dev, x_dev, encoder_fn)
            loss = criterion(outputs, labels_dev)

            # Backward and optimize
            loss.backward()
            optimizer.step()

            # Update tqdm loop
            loop.set_postfix(loss=loss.item())

        # --- Validation ---
        _model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader_param:
                val_images_dev, val_labels_dev = images.to(device), labels.to(device)
                outputs = _model(val_images_dev, omega_dev, x_dev)
                loss = criterion(outputs, val_labels_dev)
                val_loss += loss.item()

        val_loss /= len(val_loader)

        # --- Output Progress and capture stats for plotting later ---
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Validation Loss: {val_loss:.4f}")
        new_row = pd.DataFrame([{'Epoch': epoch + 1, 'Loss': loss.item(), 'Validation Loss': val_loss}])
        df = pd.concat([df, new_row], ignore_index=True)

        # --- Early Stopping ---
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                break

    # --- Finished! Plot the Training Progress ---
    display.clear_output(wait=True)
    if counter >= patience:
        print(f"Early stopping after {epoch + 1} epochs.")
    else:
        print(f"Finished training after {epoch + 1} epochs.")
    fig = px.line(df, x='Epoch', y=['Loss', 'Validation Loss'], title='Training Losses Over Epochs')
    fig.show()

    return _model


In [ ]:
from sklearn.metrics import confusion_matrix


# --- 7. Evaluation ---
def run_evaluation(trained_model: nn.Module, test_loader_param: DataLoader,
                   encoder_fn: EncoderFnType = encode_image) -> None:
    trained_model.eval()
    correct = 0
    total = 0
    omega_active_dev = omega_active.to(device)
    x_spatial_field_dev = x_spatial_field.to(device)

    # Create lists to store actual and predicted labels
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels in test_loader_param:
            images, labels = images.to(device), labels.to(device)
            outputs: torch.Tensor = trained_model(images, omega_active_dev, x_spatial_field_dev, encoder_fn)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            # noinspection PyUnresolvedReferences
            correct += (predicted == labels).sum().item()

            # Store labels and predictions
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total
    print(f"Accuracy on the test set: {accuracy:.2f}%")

    # Create confusion matrix
    confusion_df = pd.DataFrame(
        confusion_matrix(all_labels, all_predictions),
        index=[f'True {i}' for i in range(NUM_CLASSES)],
        columns=[f'Pred {i}' for i in range(NUM_CLASSES)]
    )

    # Plot confusion matrix using plotly express
    fig = px.imshow(confusion_df,
                    labels=dict(x="Predicted Label", y="True Label", color="Count"),
                    title="Confusion Matrix",
                    aspect="equal")
    fig.show()


## Training and Testing the Model
Let's hook it all up!

In [ ]:
model = run_training(train_loader, val_loader, encode_image)
run_evaluation(model, test_loader, encode_image)

## Ablation Study:

We'll create a modified version of the code where we remove the `t_spike` calculation and directly encode the pixel values using cosine and sine.

Comparing the performance of this modified version to our original model should further evidence the importance of the first spike time calculation.

In [ ]:
# noinspection PyUnusedLocal
def direct_encode(img_flat: torch.Tensor,
                  omega_active_ignored: torch.Tensor,
                  spatial_layout_ignored: torch.Tensor) -> torch.Tensor:
    """
    Directly encodes pixel values using cosine and sine functions.
    
    This implementation returns the same dimensions as encode_image calculated
    as [cos(scaled_pixels), sin(scaled_pixels)] where scaled_pixels is in the range [0, 2*pi].

    Args:
      img_flat: A flattened PyTorch tensor representing the image pixels.
      omega_active_ignored: Not used in this direct method.
      spatial_layout_ignored: Not used in this direct method.

    Returns:
      A PyTorch tensor with shape (2*N,) corresponding to the encoded image.
    """
    # Scale pixel values to the range [0, 2*pi]
    scaled_pixels = img_flat * 2 * torch.pi

    # Compute cosine and sine directly for each pixel
    cos_encoding = torch.cos(scaled_pixels)
    sin_encoding = torch.sin(scaled_pixels)

    # Concatenate to produce an encoding with 2*N dimensions
    encoded_representation = torch.cat([cos_encoding, sin_encoding])

    return encoded_representation

## Training and Testing the Default Encoder


In [ ]:
from functools import partial

wrapped_default_encode = partial(direct_encode)
model = run_training(train_loader, val_loader, wrapped_default_encode)
run_evaluation(model, test_loader, wrapped_default_encode)